In [558]:
shakespeare_data = pd.read_csv ('./Shakespeare_data.csv')
shakespeare_data = shakespeare_data.sample(1000)
shakespeare_data['PlayerLine']

46375             I have not from your eyes that gentleness
94684               [Aside]  Lord Lucius and Lucullus? hum!
45270        The enfranchisement of Arthur, whose restraint
22719      And from the mart he's somewhere gone to dinner.
74650        expostulate with her, lest her body and beauty
                                ...                        
46143        In their continuance will not feel themselves.
103966    me she did affect me: and I have heard herself...
98535                And will revolt from me to succor him.
29946                   This tribute from us, we were free:
78318             But curb it, spite of seeing. O, my lord,
Name: PlayerLine, Length: 1000, dtype: object

In [559]:
new_words = []
word_list = []
playerLines = shakespeare_data['PlayerLine']
for line in playerLines:
    new_words.append(line.split()[0])
    for word in line.split():
        word_list.append(word)

# for word in wordArray:
#     print(word)
unique_words = list(set(word_list))
unique_words.sort()
num_of_unique_words = len(unique_words)

# Get total of words
print("Total of words used in player lines:", len(wordArray))
print("Total of unique used among those words:", num_of_unique_words)

Total of words used in player lines: 14556
Total of unique used among those words: 2853


In [560]:
words = dict.fromkeys(unique_words)
index = 0
for word in words:
    words[word]=index
    index += 1

In [561]:
# Matrices of trasition probabilities of the two previous words
First_Trans = np.zeros((num_of_unique_words + 1, num_of_unique_words + 1))
Second_Trans = np.zeros((num_of_unique_words + 1, num_of_unique_words + 1))

for playerLine in playerLines:
    sentence = playerLine.split()
    for i in range(len(sentence)):
        if i < len(sentence) - 1:
            First_Trans[words[sentence[i]]][words[sentence[i + 1]]] += 1
        else:
            First_Trans[words[sentence[i]]][num_of_unique_words] += 1

        if i < len(sentence) - 2:
            Second_Trans[words[sentence[i]]][words[sentence[i + 2]]] += 1
        else:
            Second_Trans[words[sentence[i]]][num_of_unique_words] += 1

First_Trans[num_of_unique_words][num_of_unique_words] = 1
Second_Trans[num_of_unique_words][num_of_unique_words] = 1

First_Trans_Mat = First_Trans
Second_Trans_Mat = Second_Trans

for i in range(len(First_Trans)):
    First_Trans_Mat[i] = First_Trans_Mat[i] / First_Trans_Mat[i].sum()
    Second_Trans_Mat[i] = Second_Trans_Mat[i] / Second_Trans_Mat[i].sum()

unique_words.append(None)

In [563]:
# Helper Functions

# Emission probabilities
def emission(prob, state):
    for i in range(len(prob)-1):
        if state == 'Short':
            if len(unique_words[i]) < 5:
                prob[i] = prob[i] * 2
            else:
                prob[i] = prob[i] / 2
        else:
            if len(unique_words[i]) > 4:
                prob[i] = prob[i] * 2
            else:
                prob[i] = prob[i] / 2
    prob[i] = prob[i] / prob.sum()
    
    
Hiddent_State_Obj = {'Short': 0, 'Long': 1}
Hiddent_States = ['Short','Long']
Hidden_Mat = [[.8, .2],[.7, .3]]

# Returns which of the Hidden State
def nextState(Hidden_State):
    nextState = np.random.choice(Hiddent_States, size = 1, p = Hidden_Mat[Hiddent_State_Obj[Hidden_State]])
    return nextState[0]

# Hidden Makrov Model implementation
def HMM(word_choice = None, length = 7, sentence = [], hidden_State = 'Short'):
    if word_choice is None:
        word_choice = np.random.choice(new_words, size=1)[0]
    if sentence == []:
        sentence.append(word_choice)   
    next_word = np.random.choice(unique_words, size = 1,p = First_Trans_Mat[words[sentence[-1]]])[0]
    if length > 1:
        while(next_word is None):
            next_word = np.random.choice(unique_words, size = 1,p = First_Trans_Mat[words[sentence[-1]]])[0]

    while next_word is not None:
        sentence.append(next_word)
        next_Probs = First_Trans_Mat[words[sentence[-1]]] * (Second_Trans_Mat[words[sentence[-2]]]) + First_Trans_Mat[words[sentence[-1]]]/4
        next_prob = emission(next_Probs, hidden_State)
        next_Probs[-1] += 0.00001
        next_Probs = next_Probs / next_Probs.sum()
        
        if len(sentence) < length - 1:
            if next_Probs.sum() > next_Probs[-1]:
                next_Probs[-1] = next_Probs[-1] / 10
            next_Probs = next_Probs / next_Probs.sum()
            
        if len(sentence) > length + 1:
            next_Probs[-1] = next_Probs[-1] * 2
            next_Probs = next_Probs / next_Probs.sum()
        next_word = np.random.choice(unique_words, size = 1,p = next_Probs)[0]
        
        hidden_State = nextState(hidden_State)
        
    return sentence

In [564]:
sentence = "That's the letter "
generatedSentence = HMM(word_choice = sentence.split()[-1], sentence = sentence.split())
print(' '.join(str(x) for x in generatedSentence if x is not None))

That's the letter come from the skin of a pair of our general?
